In [1]:
import numpy as np
import warnings

with warnings.catch_warnings(action="ignore", category=DeprecationWarning):
    # suppress warning about pyarrow as future required dependency
    import pandas as pd

from power_grid_model import LoadGenType
from power_grid_model import PowerGridModel
from power_grid_model import initialize_array

# node
node = initialize_array("input", "node", 3)
node["id"] = [1, 2, 6]
node["u_rated"] = [10.5e3, 10.5e3, 10.5e3]

# line
line = initialize_array("input", "line", 3)
line["id"] = [3, 5, 8]
line["from_node"] = [1, 2, 1]
line["to_node"] = [2, 6, 6]
line["from_status"] = [1, 1, 1]
line["to_status"] = [1, 1, 1]
line["r1"] = [0.25, 0.25, 0.25]
line["x1"] = [0.2, 0.2, 0.2]
line["c1"] = [10e-6, 10e-6, 10e-6]
line["tan1"] = [0.0, 0.0, 0.0]
line["i_n"] = [1000, 1000, 1000]

# load
sym_load = initialize_array("input", "sym_load", 2)
sym_load["id"] = [4, 7]
sym_load["node"] = [2, 6]
sym_load["status"] = [1, 1]
sym_load["type"] = [LoadGenType.const_power, LoadGenType.const_power]
sym_load["p_specified"] = [20e6, 10e6]
sym_load["q_specified"] = [5e6, 2e6]

# source
source = initialize_array("input", "source", 1)
source["id"] = [10]
source["node"] = [1]
source["status"] = [1]
source["u_ref"] = [1.0]

# all
input_data = {"node": node, "line": line, "sym_load": sym_load, "source": source}

In [2]:
from power_grid_model.utils import json_serialize_to_file
import tempfile
from pathlib import Path

temp_path = Path(tempfile.gettempdir())
json_serialize_to_file(temp_path / "input.json", input_data)

In [3]:
with open(temp_path / "input.json", "r") as f:
    print(f.read())

{
  "version": "1.0",
  "type": "input",
  "is_batch": false,
  "attributes": {},
  "data": {
    "node": [
      {"id": 1, "u_rated": 10500},
      {"id": 2, "u_rated": 10500},
      {"id": 6, "u_rated": 10500}
    ],
    "line": [
      {"id": 3, "from_node": 1, "to_node": 2, "from_status": 1, "to_status": 1, "r1": 0.25, "x1": 0.2, "c1": 1e-05, "tan1": 0, "i_n": 1000},
      {"id": 5, "from_node": 2, "to_node": 6, "from_status": 1, "to_status": 1, "r1": 0.25, "x1": 0.2, "c1": 1e-05, "tan1": 0, "i_n": 1000},
      {"id": 8, "from_node": 1, "to_node": 6, "from_status": 1, "to_status": 1, "r1": 0.25, "x1": 0.2, "c1": 1e-05, "tan1": 0, "i_n": 1000}
    ],
    "sym_load": [
      {"id": 4, "node": 2, "status": 1, "type": 0, "p_specified": 20000000, "q_specified": 5000000},
      {"id": 7, "node": 6, "status": 1, "type": 0, "p_specified": 10000000, "q_specified": 2000000}
    ],
    "source": [
      {"id": 10, "node": 1, "status": 1, "u_ref": 1}
    ]
  }
}


In [4]:
# round trip and run power flow
from power_grid_model.utils import json_deserialize_from_file

imported_data = json_deserialize_from_file(temp_path / "input.json")

pgm = PowerGridModel(imported_data)
result = pgm.calculate_power_flow()

print(pd.DataFrame(result["node"]))

   id  energized      u_pu             u   u_angle             p             q
0   1          1  0.998988  10489.375043 -0.003039  3.121451e+07  6.991358e+06
1   2          1  0.952126   9997.325181 -0.026031 -2.000000e+07 -5.000000e+06
2   6          1  0.962096  10102.012975 -0.021895 -1.000000e+07 -2.000000e+06
